# 1. Dataset cell type pseudobulk assembly

Load in datasets. Check availability of raw data. Construct pseudobulks.

In [1]:
import matplotlib.pyplot as plt
import scanpy as sc
import numpy as np
import time

In [2]:
import decoupler as dc
import pandas as pd
import sys
import anndata
import pickle

In [3]:
ads = {}

In [4]:
rt = '/home/ec2-user/curation/'
filenames = {'amrute':'ad.h5ad','brener':'ad.h5ad','chaffin':'human_dcm_hcm_scportal_03.17.2022.h5ad','eraslan':'ad.h5ad','hill':'ad.h5ad',
            'litvinukova':'ad.h5ad','kanemaru':'ad.h5ad','kuppe':'ad.h5ad','reichart':'ad.h5ad','selewa':'ad.h5ad',
             'knight_schrijver':'ad.h5ad',
            'sim':'ad.h5ad','simonson':'ICM_scportal_05.24.2022.h5ad','tucker':'healthy_human_4chamber_map_unnormalized_V4.h5ad',
            'linna_kuosmanen':['carebank.h5ad','periheart.h5ad']}

neuron_cts = ['Glia','glia','neuron','Neuron','neural','Neural','Schwann']

## Amrute

In [5]:
dataset = 'amrute'
ad = sc.read_h5ad(rt+dataset+'/'+filenames[dataset])

In [6]:
print(ad.X.max())
print(ad.raw.X.max())

8.7418131753239
5099.0


In [7]:
ad.obs.iloc[0]

orig.ident                TWCM-190-R-post
nCount_RNA                         9235.0
nFeature_RNA                         3069
percent.mt                        0.06497
scrublet_score                   0.142857
scrublet_cluster_score           0.158129
bh_pval                          0.028352
nCount_SCT                         2364.0
nFeature_SCT                         1286
SCT_snn_res.0.3                        13
seurat_clusters                        24
cell.type                       Adipocyte
condition                           Rpost
RNA_snn_res.0.2                         0
RNA_snn_res.0.3                         0
RNA_snn_res.0.4                         2
RNA_snn_res.0.5                        NA
RNA_snn_res.0.6                        NA
Fibro_z                               NaN
Pericyte_z                            NaN
SMC_z                                 NaN
SCT_snn_res.0.1                         6
SCT_snn_res.0.2                        10
SCT_snn_res.0.4                   

In [8]:
ad.obs.rename(columns={'orig.ident':'donor_id','cell.type':'cell_type'},inplace=True)

In [9]:
ad.obs['donor_id'].unique()

array(['TWCM-190-R-post', 'TWCM-229-R-post', 'TWCM-229-R-pre',
       'TWCM-239-R-post', 'TWCM-296-R-post', 'TWCM-296-R-pre',
       'TWCM-359-NR-post', 'TWCM-359-NR-pre', 'TWCM-363-NR-post',
       'TWCM-363-NR-pre', 'TWCM-373-NR-post', 'TWCM-373-NR-pre',
       'TWCM-388-NR-pre', 'TWCM-397-NR-post', 'TWCM-397-NR-pre',
       'TWCM-410-NR-post', 'TWCM-410-NR-pre', 'TWCM-463-R-post',
       'TWCM-463-R-pre', 'TWCM-11-103', 'TWCM-11-41', 'TWCM-11-74',
       'TWCM-11-78', 'TWCM-11-82', 'TWCM-13-152', 'TWCM-13-168',
       'TWCM-13-192', 'TWCM-13-80', 'TWCM-14-173', 'TWCM-190-R-pre',
       'TWCM-239-R-pre', 'TWCM-376-NR-post', 'TWCM-376-NR-pre',
       'TWCM-378-NR-post', 'TWCM-378-NR-pre', 'TWCM-388-NR-post',
       'H_ZC-11-292', 'TWCM-11-192', 'TWCM-13-104', 'TWCM-13-1'],
      dtype=object)

In [10]:
ad = ad[ad.obs['condition']!='Donor'] #donors are from Koenig

Sexes and ages are not marked in the Seurat object.

In [11]:
ad.obs['sex']=None
female_donors = ['TWCM-'+x for x in ['13-1','13-104','14-173','296','370','397']]
filt = ad.obs['donor_id'].str.contains( "|".join(female_donors),regex=True)
ad.obs.loc[filt,'sex']='female'
ad.obs.loc[~filt,'sex']='male'

/tmp/ipykernel_23135/3479450247.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['sex']=None


In [12]:
age_map = {'190':18,'229':60,'239':44,'296':60,
           '359':46,'363':64,'370':67,'371':61,
           '373':69,'376':37,'397':47,'410':58,
           '463':29,
           '388':1000,'378':1001} #placeholders

In [13]:
ad.obs['age'] = ad.obs['donor_id'].str.split('-').str[1].map(age_map)

In [14]:
ad.obs['age'].value_counts(dropna=False)

age
60      16557
58      14547
29      12651
37      11392
1000     8533
18       8397
46       6371
47       6334
44       5486
1001     5292
69       3753
64       3550
Name: count, dtype: int64

TWCM-388 and TWCM-378 do not show up in metadata. Are these TWCM-370 and TWCM-371???

388: nonresponder, male.

378: nonresponder, male.

370 is female and 371 is male.

The identities are unknown at a cursory examination.

In [15]:
dids = ad.obs['donor_id'].str.split('-').str[1].unique()
metakeys = np.array(list(age_map.keys()))
print(dids[~np.isin(dids,metakeys)])
print(metakeys[~np.isin(metakeys,dids)])

[]
['370' '371']


In [16]:
ad.obs

,donor_id,nCount_RNA,nFeature_RNA,percent.mt,scrublet_score,scrublet_cluster_score,bh_pval,nCount_SCT,nFeature_SCT,SCT_snn_res.0.3,...,RNA_snn_res.0.6,Fibro_z,Pericyte_z,SMC_z,SCT_snn_res.0.1,SCT_snn_res.0.2,SCT_snn_res.0.4,SCT_snn_res.0.5,sex,age
TWCM-190-R-post_AACCATGGTTACTGAC,TWCM-190-R-post,9235.0,3069,0.064970,0.142857,0.158129,0.028352,2364.0,1286,13,...,NA,NaN,NaN,NaN,6,10,14,17,male,18
TWCM-190-R-post_AACGTTGTCTGAAAGA,TWCM-190-R-post,4863.0,2183,0.041127,0.185930,0.158129,0.028352,2849.0,1864,13,...,NA,NaN,NaN,NaN,6,10,14,17,male,18
TWCM-190-R-post_CCATGTCTCAGAGCTT,TWCM-190-R-post,5478.0,2482,0.036510,0.166667,0.158129,0.028352,2742.0,1828,13,...,NA,NaN,NaN,NaN,6,10,14,17,male,18
TWCM-190-R-post_CGAGCACTCAGTGTTG,TWCM-190-R-post,2532.0,1390,0.236967,0.175904,0.158129,0.028352,2366.0,1390,13,...,NA,NaN,NaN,NaN,6,10,14,17,male,18
TWCM-190-R-post_GCTCTGTGTGTTTGGT,TWCM-190-R-post,8252.0,2673,0.000000,0.185930,0.158129,0.028352,2420.0,1221,13,...,NA,NaN,NaN,NaN,6,10,14,17,male,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TWCM-463-R-pre_TTGCCGTAGGACCACA,TWCM-463-R-pre,1440.0,1056,0.625000,0.011727,0.018757,0.971327,1614.0,1056,19,...,NA,NaN,NaN,NaN,11,17,23,24,male,29
TWCM-463-R-pre_TTGCCGTCAATAGAGT,TWCM-463-R-pre,1608.0,1162,1.119403,0.030303,0.017241,0.971327,1681.0,1162,19,...,NA,NaN,NaN,NaN,11,17,23,24,male,29
TWCM-463-R-pre_TTGTAGGGTAGCGATG,TWCM-463-R-pre,1224.0,892,0.816993,0.092784,0.013029,0.973777,1552.0,893,19,...,NA,NaN,NaN,NaN,11,17,23,24,male,29
TWCM-463-R-pre_TTTGGTTCAGCCTTTC,TWCM-463-R-pre,1256.0,891,0.238854,0.007519,0.013029,0.973777,1556.0,892,19,...,NA,NaN,NaN,NaN,11,17,23,24,male,29


In [17]:
ad = dc.get_pseudobulk(ad,sample_col='donor_id',groups_col='cell_type',
    mode='sum',
    min_cells=0,
    min_counts=0,use_raw=True)


In [18]:
ad.var.set_index('features',inplace=True)

In [19]:
ads[dataset] = ad.copy()

## Brener

The annotated data were removed from Broad. Also, the Broad data did not have counts.

In [20]:
dataset = 'brener'
ad = sc.read_h5ad(rt+dataset+'/'+filenames[dataset])
ad_geo = sc.read_h5ad(rt+dataset+'/ad_geo.h5ad')

In [21]:
ad_geo.obs.iloc[0]

doublet_scores                  0.152416
barcode               AACCTGATCCAATGCA-1
exon_ratio                      0.758098
batches                      8_144391-RV
Virus                           Covid_MT
n_genes                             4843
n_genes_by_counts                   4843
total_counts                     22781.0
total_counts_mt                    384.0
pct_counts_mt                   1.685615
fail_exonratio                     False
predicted_doublets                 False
Name: AACCTGATCCAATGCA-1-0-0-0-0-0-0-0-0-0-0-0-0, dtype: object

In [22]:
ad = ad[ad.obs['Virus']=='Covid']

Make sure all the dimensions match.

In [23]:
assert all(ad_geo.var_names == ad.var_names)

In [24]:
assert all(ad_geo.obs_names == ad.obs_names)

In [25]:
print(ad_geo.X.max())
# print(ad_geo.raw.X.max())

print(ad.X.max())
print(ad.raw.X.max())

10553.0
8.149373
8.149373


In [26]:
ad_raw = ad.raw.to_adata()
ad_raw.X = ad_geo.X
ad.raw=ad_raw

In [27]:
print(ad_geo.X.max())

print(ad.X.max())
print(ad.raw.X.max())

10553.0
8.149373
10553.0


In [28]:
ad.obs.rename(columns={'batches':'donor_id','leiden':'cell_type'},inplace=True)

In [29]:
ad.obs['sex']=None
filt = ad.obs['donor_id']=='15_144548-RV' #the one female sample
ad.obs.loc[filt,'sex']='female'
ad.obs.loc[~filt,'sex']='male'

In [30]:
ad.obs.iloc[0]

doublet_scores                  0.152416
predicted_doublets                 False
barcode               AACCTGATCCAATGCA-1
exon_ratio                      0.758098
donor_id                     8_144391-RV
Virus                              Covid
n_genes                             4843
n_genes_by_counts                   4843
total_counts                     22781.0
total_counts_mt                    384.0
pct_counts_mt                   1.685615
cell_type                     Macrophage
Condition                   Microthrombi
sex                                 male
Name: AACCTGATCCAATGCA-1-0-0-0-0-0-0-0-0-0-0-0-0, dtype: object

The publication does not actually disclose how the GEO/AnnData entries map to "Study ID" identifiers. So we use table ST6, final column.

In [31]:
ad.obs['donor_id'].value_counts()

donor_id
11_144422-RV    9947
15_144548-RV    7840
8_144391-RV     7208
13_144471-RV    6851
10_144421-RV    5787
16_144608-RV    4014
10_144396-RV    1846
Name: count, dtype: int64

In [32]:
ad.obs['donor_id'].str.split('-')

AACCTGATCCAATGCA-1-0-0-0-0-0-0-0-0-0-0-0-0     [8_144391, RV]
GGGATCCCAACACGTT-1-0-0-0-0-0-0-0-0-0-0-0-0     [8_144391, RV]
ACGTAACCATTACGGT-1-0-0-0-0-0-0-0-0-0-0-0-0     [8_144391, RV]
TCCCACACACGAGGAT-1-0-0-0-0-0-0-0-0-0-0-0-0     [8_144391, RV]
CTCACTGTCGAGATAA-1-0-0-0-0-0-0-0-0-0-0-0-0     [8_144391, RV]
                                                   ...       
AAGGAATAGCGTGTCC-1-1-0-0-0-0-0-0              [16_144608, RV]
CAGCAGCTCTGCGTCT-1-1-0-0-0-0-0-0              [16_144608, RV]
AGGAGGTCATATGCGT-1-1-0-0-0-0-0-0              [16_144608, RV]
TAGGGTTGTGTGTGTT-1-1-0-0-0-0-0-0              [16_144608, RV]
CCGTAGGGTACAAGCG-1-1-0-0-0-0-0-0              [16_144608, RV]
Name: donor_id, Length: 43493, dtype: object

In [33]:
study_id_map = {'11_144422':'19','15_144548':'66','8_144391':'05',
                '13_144471':'45','10_144421':'51','16_144608':'61',
                '10_144396':'39'}

age_map = {'05':83,'39':71,'61':58,'19':68,'45':65,'51':63,'66':69}
ad.obs['age'] = ad.obs['donor_id'].str.split('-').str[0].map(study_id_map).map(age_map)

In [34]:
ad = dc.get_pseudobulk(ad,sample_col='donor_id',groups_col='cell_type',
    mode='sum',
    min_cells=0,
    min_counts=0,use_raw=True)


In [35]:
ads[dataset] = ad.copy()

## Eraslan

In [36]:
dataset = 'eraslan'
ad = sc.read_h5ad(rt+dataset+'/'+filenames[dataset])

In [37]:
ad=ad[ad.obs['tissue']=='anterior wall of left ventricle']

In [38]:
print(ad.X.max())
print(ad.raw.X.max())

8.578069
9240.0


In [39]:
ad = dc.get_pseudobulk(ad,sample_col='donor_id',groups_col='cell_type',
    mode='sum',
    min_cells=0,
    min_counts=0,use_raw=True)

In [40]:
ad.obs['Age_bin'].value_counts()

Age_bin
61-70    32
41-50    16
Name: count, dtype: int64

This is incorrect. Per Table S1, GTEX-1ICG6 is 70-79.

In [41]:
age_map = {'GTEX-1ICG6':'70-79','GTEX-13N11':'50-59','GTEX-15RIE':'60-69'}
ad.obs['Age_bin'] = ad.obs['donor_id'].map(age_map)

In [42]:
ad.obs['Age_bin'].value_counts()

Age_bin
50-59    16
60-69    16
70-79    16
Name: count, dtype: int64

In [43]:
ad.var.set_index('feature_name',inplace=True)

In [44]:
ads[dataset] = ad.copy()

## Hill

This dataset was previously assembled manually from flat tables.

In [45]:
dataset = 'hill'
ad = sc.read_h5ad(rt+dataset+'/'+filenames[dataset])

In [46]:
ad.obs.rename(columns={'MainCellType':'cell_type'},inplace=True)

In [47]:
print(ad.X.max())

20720.0


In [48]:
len(ad.obs['donor_id'].unique())

14

Evidently 13_198 is duplicated (LV and RV), and UK1/2 somehow map to 3B62D and FC3CB.

In [49]:
for x in ad.obs['donor_id'].unique().sort_values(): print(x)

13_198_LV
13_198_RV
13_235
P8
P26
P28
P33
P36
P40
P64
P75
P86
UK1
UK2


In [50]:
ad.obs['age'].value_counts()

age
11y           36866
11y_3m_12d    21850
4y_4m_20d     16321
16y_2m_25d    15070
3y_3m_17d     14055
17y_3m_13d    12906
3y_9m_3d      12236
0y_7m_3d       7213
0y_2m_3d       6995
0y_8m_9d       6195
9y_7m_6d       5158
0y_3m_3d       2428
Name: count, dtype: int64

In [51]:
ad = dc.get_pseudobulk(ad,sample_col='donor_id',groups_col='cell_type',
    mode='sum',
    min_cells=0,
    min_counts=0,use_raw=False)

In [52]:
ad.obs[['donor_id','age']].drop_duplicates()

,donor_id,age
13_198_RV_Adipo,13_198_RV,11y
13_235_Adipo,13_235,11y
P40_Adipo,P40,4y_4m_20d
P64_Adipo,P64,3y_3m_17d
P75_Adipo,P75,16y_2m_25d
P86_Adipo,P86,11y_3m_12d
UK1_Adipo,UK1,9y_7m_6d
UK2_Adipo,UK2,3y_9m_3d
13_198_LV_CF,13_198_LV,11y
P26_CF,P26,0y_3m_3d


Clearly FC3CB is UK1 and 3B62D is UK2.

In [53]:
ad.obs['age'] = ad.obs['age'].str.split('y').str[0].astype(float)

In [54]:
ads[dataset] = ad.copy()

## Kanemaru

In [55]:
dataset = 'kanemaru'
ad = sc.read_h5ad(rt+dataset+'/'+filenames[dataset])

In [56]:
print(ad.X.max())
print(ad.raw.X.max())

9.030837
40290.0


In [57]:
ad=ad[ad.obs['is_primary_data']]

Some donors got resequenced, and D11 was erroneously added into primary data. Per Fig. 1a, there should be 211,060 newly-generated RNA data points.

In [58]:
print(ad.shape)
ad.obs['donor_id'].value_counts()

(211061, 32583)


donor_id
AH1     55785
A61     34607
D8      33554
AH2     27849
AV13    18886
D7      13925
AV14    10624
AV3      7388
AV10     5757
D3       2685
D11         1
Name: count, dtype: int64

In [59]:
ad = ad[ad.obs['donor_id']!='D11']

In [60]:
# ad.obs['Age_bin'] = ad.obs['age']

In [61]:
ad.obs.rename(columns={'age':'age_decade'},inplace=True)

AH2 is reported as 45-50 in Supp. Table but 40-45 in AnnData.  Numbers match otherwise.

In [62]:
# ad.obs

In [63]:
age_map = {'A61':'70-75','AH1':'45-50','AH2':'45-50',
           'AV10':'20-25','AV13':'70-75','AV14':'45-50',
           'AV3':'60-65','D3':'55-60','D7':'60-65',
           'D8':'45-50'}
           
ad.obs['Age_bin'] = ad.obs['donor_id'].map(age_map)

/tmp/ipykernel_23135/395475291.py:6: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['Age_bin'] = ad.obs['donor_id'].map(age_map)


In [64]:
ad = dc.get_pseudobulk(ad,sample_col='donor_id',groups_col='cell_type',
    mode='sum',
    min_cells=0,
    min_counts=0,use_raw=True)

In [65]:
ad.obs[['donor_id','Age_bin']].drop_duplicates()

,donor_id,Age_bin
A61_adipocyte,A61,70-75
AH1_adipocyte,AH1,45-50
AH2_adipocyte,AH2,45-50
AV10_adipocyte,AV10,20-25
AV13_adipocyte,AV13,70-75
AV14_adipocyte,AV14,45-50
AV3_adipocyte,AV3,60-65
D3_adipocyte,D3,55-60
D7_adipocyte,D7,60-65
D8_adipocyte,D8,45-50


In [66]:
ad.var.set_index('feature_name',inplace=True)

In [67]:
ads[dataset] = ad.copy()

## Knight-Schrijver

In [68]:
dataset = 'knight_schrijver'
ad = sc.read_h5ad(rt+dataset+'/'+filenames[dataset])

In [69]:
print(ad.X.max())
print(ad.raw.X.max())

8.866902
8476.0


In [70]:
ad=ad[ad.obs['is_primary_data']]

In [71]:
ad.obs['age.days.GA'].value_counts()

age.days.GA
80.0    8332
64.0    5950
69.0    4429
58.0    4083
66.0    3391
61.0    3239
67.0    1465
Name: count, dtype: int64

In [72]:
# ad.obs['age'] = ad.obs['age.days.GA'].astype(float)
ad.obs['age'] = 0 #all fetal

/tmp/ipykernel_23135/2539002409.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['age'] = 0 #all fetal


In [73]:
ad = dc.get_pseudobulk(ad,sample_col='donor_id',groups_col='cell_type',
    mode='sum',
    min_cells=0,
    min_counts=0,use_raw=True)

In [74]:
ad.var.set_index('feature_name',inplace=True)

In [75]:
ads[dataset] = ad.copy()

## Kuppe

In [76]:
dataset = 'kuppe'
ad = sc.read_h5ad(rt+dataset+'/'+filenames[dataset])

In [77]:
print(ad.X.max())
print(ad.raw.X.max())

8.693713
21324.0


In [78]:
ad.obs['age'] = ad.obs['development_stage'].str.split('-').str[0].astype(float)

In [79]:
ad = dc.get_pseudobulk(ad,sample_col='donor_id',groups_col='cell_type',
    mode='sum',
    min_cells=0,
    min_counts=0,use_raw=True)

In [80]:
ad.var.set_index('feature_name',inplace=True)

In [81]:
ads[dataset] = ad.copy()

## Linna-Kuosmanen

In [82]:
dataset = 'linna_kuosmanen'
ad1 = sc.read_h5ad(rt+dataset+'/'+filenames[dataset][0])
ad2 = sc.read_h5ad(rt+dataset+'/'+filenames[dataset][1])

In [83]:
assert ad1.obs['is_primary_data'].mean()==1
assert ad2.obs['is_primary_data'].mean()==1

In [84]:
ad1.var.set_index('feature_name',inplace=True)
ad2.var.set_index('feature_name',inplace=True)

In [85]:
ad = anndata.concat((ad1,ad2),axis=0)

/home/ec2-user/miniconda/lib/python3.9/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")
/home/ec2-user/miniconda/lib/python3.9/site-packages/anndata/_core/anndata.py:1897: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


In [86]:
ad.obs_names_make_unique()

In [87]:
print(ad.X.max())
print(ad.raw.X.max())

2588.5754
2577.0


In [88]:
age_map = {'pediatric stage':'0-9','postnatal stage':'10-19','third decade stage':'20-29',
           'fourth decade stage':'30-39','fifth decade stage':'40-49','sixth decade stage':'50-59',
           'seventh decade stage':'60-69','eighth decade stage':'70-79','ninth decade stage':'80-89'}

In [89]:
ad.obs['development_stage'].value_counts(dropna=False)

development_stage
seventh decade stage    246548
eighth decade stage     166929
sixth decade stage      106508
fifth decade stage       46551
ninth decade stage       46526
Name: count, dtype: int64

In [90]:
ad.obs['Age_bin']  = ad.obs['development_stage'].map(age_map)

In [91]:
ad = dc.get_pseudobulk(ad,sample_col='donor_id',groups_col='cell_type',
    mode='sum',
    min_cells=0,
    min_counts=0,use_raw=True)

In [92]:
ads[dataset] = ad.copy()

## Litvinukova

In [93]:
dataset = 'litvinukova'
ad = sc.read_h5ad(rt+dataset+'/'+filenames[dataset])

In [94]:
print(ad.X.max())
print(ad.raw.X.max())

7.4499817
9386.0


Bins in the AnnData are coarser than in Supplementary Table 1.

In [95]:
age_map = {'D1':'50-55','D2':'60-65','D3':'55-60',
           'D4':'70-75','D5':'65-70','D6':'70-75',
           'D7':'60-65','D11':'60-65','H2':'50-55',
           'H3':'50-55','H4':'55-60','H5':'50-55',
           'H6':'40-45','H7':'45-50'}
           
ad.obs['Age_bin'] = ad.obs['donor_id'].map(age_map)

In [96]:
ad = dc.get_pseudobulk(ad,sample_col='donor_id',groups_col='cell_type',
    mode='sum',
    min_cells=0,
    min_counts=0,use_raw=True)

In [97]:
ad.var.set_index('feature_name',inplace=True)

In [98]:
ads[dataset] = ad.copy()

## Reichart

In [99]:
dataset = 'reichart'
ad = sc.read_h5ad(rt+dataset+'/'+filenames[dataset])

In [100]:
print(ad.X.max())
print(ad.raw.X.max())

8.956366
8499.0


In [101]:
ad=ad[ad.obs['is_primary_data']]

In [102]:
age_map = {'pediatric stage':'0-9','postnatal stage':'10-19','third decade stage':'20-29',
           'fourth decade stage':'30-39','fifth decade stage':'40-49','sixth decade stage':'50-59',
           'seventh decade stage':'60-69','eighth decade stage':'70-79'}

In [103]:
ad.obs['development_stage'].value_counts()

development_stage
fifth decade stage      199539
seventh decade stage    151949
sixth decade stage      118880
fourth decade stage      84930
postnatal stage          42152
third decade stage       41770
pediatric stage          18886
eighth decade stage       7849
Name: count, dtype: int64

In [104]:
ad.obs['Age_bin']  = ad.obs['development_stage'].map(age_map)

/tmp/ipykernel_23135/1655173530.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['Age_bin']  = ad.obs['development_stage'].map(age_map)


In [105]:
ad = dc.get_pseudobulk(ad,sample_col='donor_id',groups_col='cell_type',
    mode='sum',
    min_cells=0,
    min_counts=0,use_raw=True)

In [106]:
ad.var.set_index('feature_name',inplace=True)

In [107]:
ads[dataset] = ad.copy()

## Selewa

In [108]:
dataset = 'selewa'
ad = sc.read_h5ad(rt+dataset+'/'+filenames[dataset])

In [109]:
print(ad.X.max())
print(ad.raw.X.max())

8.325382
3930.0


In [110]:
ad=ad[ad.obs['is_primary_data']]

In [111]:
ad.obs['age'] = ad.obs['development_stage'].str.split('-').str[0].astype(float)

/tmp/ipykernel_23135/1463941940.py:1: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['age'] = ad.obs['development_stage'].str.split('-').str[0].astype(float)


In [112]:
ad = dc.get_pseudobulk(ad,sample_col='donor_id',groups_col='cell_type',
    mode='sum',
    min_cells=0,
    min_counts=0,use_raw=True)

In [113]:
ad.var.set_index('feature_name',inplace=True)

In [114]:
ads[dataset] = ad.copy()

## Sim

In [115]:
dataset = 'sim'
ad = sc.read_h5ad(rt+dataset+'/'+filenames[dataset])

In [116]:
print(ad.X.max())
print(ad.raw.X.max())

8.49085196676018
28948.0


In [117]:
ad=ad[ad.obs['is_primary_data']]

Parse fetal data and set ages to zero.

In [118]:
filt = ~ad.obs['sample_id'].str.startswith('fetal')
ad.obs['age'] = None
ad.obs.loc[filt,'age'] = ad.obs.loc[filt,'development_stage'].str.split('-').str[0].astype(float)
ad.obs.loc[~filt,'age'] = 0

/tmp/ipykernel_23135/3547689480.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  ad.obs['age'] = None


In [119]:
ad = dc.get_pseudobulk(ad,sample_col='donor_id',groups_col='cell_type',
    mode='sum',
    min_cells=0,
    min_counts=0,use_raw=True)

In [120]:
ad.var.set_index('feature_name',inplace=True)

In [121]:
ads[dataset] = ad.copy()

## Simonson

In [122]:
dataset = 'simonson'
ad = sc.read_h5ad(rt+dataset+'/'+filenames[dataset])

In [123]:
print(ad.X.max())

2981.0


In [124]:
ad.obs.rename(columns={'cell_type_leiden0.5':'cell_type'},inplace=True)

In [125]:
ad = dc.get_pseudobulk(ad,sample_col='donor_id',groups_col='cell_type',
    mode='sum',
    min_cells=0,
    min_counts=0,use_raw=False)

In [126]:
ads[dataset] = ad.copy()

## Tucker

In [127]:
dataset = 'tucker'
ad = sc.read_h5ad(rt+dataset+'/'+filenames[dataset])

In [128]:
print(ad.X.max())

5365.0


In [129]:
ad.obs.rename(columns={'Cluster':'cell_type','biological.individual':'donor_id'},inplace=True)

In [130]:
for x in ad.obs['donor_id'].unique(): print(x)

1600
1666
1681
1702
1708
1723
1221


Sexes and ages need to be read off Table 1 manually.

In [131]:
ad.obs['sex']=None
female_donors = ['1221','1600','1708','1723']
filt = ad.obs['donor_id'].str.contains( "|".join(female_donors),regex=True)
ad.obs.loc[filt,'sex']='female'
ad.obs.loc[~filt,'sex']='male'

In [132]:
age_map = {'1221':52,'1600':51,'1666':54,'1681':39,'1702':59,'1708':60,'1723':47}
ad.obs['age'] = ad.obs['donor_id'].map(age_map)

In [133]:
ad = dc.get_pseudobulk(ad,sample_col='donor_id',groups_col='cell_type',
    mode='sum',
    min_cells=0,
    min_counts=0,use_raw=False)

In [134]:
ads[dataset] = ad.copy()

# Export

Check that the pseudobulks are correct (integer-valued).

In [135]:
for x in ads.keys():
    print(x)
    assert ((ads[x].X%1)==0).all()

amrute
brener
eraslan
hill
kanemaru
knight_schrijver
kuppe
linna_kuosmanen
litvinukova
reichart
selewa
sim
simonson
tucker


In [136]:
with open('/home/ec2-user/curation/heart_datasets/gg_250327_heart_celltype_psbulks.p', 'wb') as fp:
    pickle.dump(ads, fp)


In [141]:
!md5sum /home/ec2-user/curation/heart_datasets/gg_250327_heart_celltype_psbulks.p

9354a2445083ab9ae889bb38781337a0  /home/ec2-user/curation/heart_datasets/gg_250327_heart_celltype_psbulks.p
